In [1]:
from selenium.webdriver import Firefox
from selenium.webdriver.common.by import By

# Provisórios
from time import sleep

# Page Objects

In [2]:
from abc import ABC


class SeleniumObject:
    def find_element(self, locator):
        return self.webdriver.find_element(*locator)

    def find_elements(self, locator):
        return self.webdriver.find_elements(*locator)

class Page(ABC, SeleniumObject):
    def __init__(self, webdriver, url=''):
        self.webdriver = webdriver
        self.url = url
        self._reflection()
    
    def open(self):
        self.webdriver.get(self.url)

    def _reflection(self):
        for atributo in dir(self):
            atributo_real = getattr(self, atributo)
            if isinstance(atributo_real, PageElement):
                atributo_real.webdriver = self.webdriver


class PageElement(ABC, SeleniumObject):
    def __init__(self, webdriver=None):
        self.webdriver = webdriver

# Pages

## Pages

In [3]:
class Login(PageElement):
    login_input = (By.ID, 'user-name')
    password_input = (By.ID, 'password')
    submit_button = (By.CSS_SELECTOR, '[type="submit"]')

    def logar(self, usuario, senha):
        self.find_element(self.login_input).send_keys(usuario)
        self.find_element(self.password_input).send_keys(senha)
        self.find_element(self.submit_button).click()


class Planejamento(PageElement):
    spg_button = (By.CSS_SELECTOR, '.ng-tns-c37-10')

    def acessarSpg(self):
        self.find_element(self.spg_button).click()


class Terceirizados(PageElement):

    loc_funcionarios = (By.CSS_SELECTOR, 'tbody > tr')

    def funcionarios_cadastrados(self):
        funcionarios = self.find_elements(self.loc_funcionarios)

        po_funcionarios = []
        for funcionario in funcionarios:
            po_funcionarios.append(Funcionario(funcionario))

        return po_funcionarios


class DemaisInformacoes(PageElement):
    ...


class Funcionario():
    def __init__(self, selenium_object):
        """
        Funcionario contem as informações do Terceirizados.
        selenium_object: objeto te a linha na tabela a qual
        o funcionario pertence.
        """
        self.selenium_object = selenium_object
        # Informações
        self.nome = (By.CSS_SELECTOR, 'div > span')
        self.cpf = (By.CSS_SELECTOR, 'span > span')
        self.dias_trabalhados = None
        self.salario_base = None
        self.salario_total = None
        # Elementos
        self._dias_trabalhados = self.dias_trabalhados
        self._salario_base = self.salario_base
        self._demais_informacoes = (By.CSS_SELECTOR, 'button')
        self._load()

    def demais_informacoes(self):
        self.selenium_object.find_element(*self._demais_informacoes).click()

    def modificar_dias_trabalhados(self, dias):
        ...

    def modificar_salario_base(self, salario):
        ...

    def _load(self):
        self.nome = self.selenium_object.find_element(
            *self.nome
        ).text

        self.cpf = self.selenium_object.find_element(
            *self.cpf
        ).text

        # inputs
        inputs = (By.CSS_SELECTOR, 'input')

        # dias trabalhados
        input_elements = self.selenium_object.find_elements(
            *inputs
        )

        self.dias_trabalhados = input_elements[0].get_attribute('value')
        self.dias_trabalhados = float(
            self.dias_trabalhados.replace('.', '').replace(',', '.')
        )
        # elemento clicável
        self._dias_trabalhados = input_elements[0]

        # salario base
        self.salario_base = input_elements[1].get_attribute('value')
        self.salario_base = float(
            self.salario_base.replace('.', '').replace(',', '.')
        )
        # elemento clicável
        self._salario_base = input_elements[1]

        # salario total
        self.salario_total = input_elements[7].get_attribute('value')
        self.salario_total = float(
            self.salario_total.replace('.', '').replace(',', '.')
        )

    def __repr__(self):
        return "ALOOOOO"
        # return f'Funcionario(nome="{self.nome}"", cpf="{self.cpf}")'


class SpecialInput():

    def input(elemento, novo_valor):
        ...


In [4]:
class PageLogin(Page):
    login = Login()


class PageGuardiao(Page):
    planejamento = Planejamento()


class PageFatura(Page):
    terceirizados = Terceirizados()
    demais_informacoes = DemaisInformacoes()


# Testes

In [5]:
with open('credenciais.txt') as f:
    login = f.readline()
    senha = f.readline()

browser = Firefox()

pagina = PageLogin(
    browser,
    'https://guardiaov4.seplag.ce.gov.br/auth'
)

pagina.open()
pagina.login.logar(login, senha)

pagina = PageGuardiao(browser)
sleep(1)
pagina.planejamento.acessarSpg()

pagina = PageFatura(browser)
